In [1]:
import numpy as np
import pandas as pd
import csv

import time

import cuml,cupy,cudf
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

In [2]:
train_path = "../input/amazon-ml-challenge/train.csv"
test_path = "../input/amazon-ml-challenge/test.csv"

train_df = cudf.read_csv(train_path,escapechar="\\",quoting=csv.QUOTE_NONE)
test_df = cudf.read_csv(test_path,escapechar="\\",quoting=csv.QUOTE_NONE)
train_df_cpu = pd.read_csv(train_path,escapechar="\\",quoting=csv.QUOTE_NONE)
test_df_cpu = pd.read_csv(test_path,escapechar="\\",quoting=csv.QUOTE_NONE)

In [6]:
# COSINE SIMILARITY
'''
model = TfidfVectorizer(stop_words='english', binary=True, max_features=128)
train_embed = model.fit_transform(train_df.TITLE).toarray()
test_embed = model.transform(test_df.TITLE).toarray()
BATCH_SIZE = 256
test_pred = []
NUM_BATCHES = test_embed.shape[0]//BATCH_SIZE

start = time.time()
for i in range(NUM_BATCHES):
    start = i*BATCH_SIZE
    end = (i+1)*BATCH_SIZE
    sim = cupy.matmul(train_embed,test_embed[start:end].T).T
    
    for j in range(end-start):
        ids = cupy.where(sim[j,] > 0.8)[0]
        
        ls = train_df_cpu.iloc[cupy.asnumpy(ids)].BROWSE_NODE_ID.values
        test_pred.append(ls)
    if(i % 50 == 0):
        print("DONE PRODUCTS :", end)
        print("Time : ",int(time.time()-start),"sec")'''
print("COSINE SIMILARITY")

DONE PRODUCTS : 256
Time :  1627650053.2430296 sec
DONE PRODUCTS : 13056
Time :  1627637314.8024905 sec
DONE PRODUCTS : 25856
Time :  1627624576.6784413 sec
DONE PRODUCTS : 38656
Time :  1627611838.7348804 sec
DONE PRODUCTS : 51456
Time :  1627599099.9838512 sec
DONE PRODUCTS : 64256
Time :  1627586361.3189197 sec
DONE PRODUCTS : 77056
Time :  1627573623.5454156 sec
DONE PRODUCTS : 89856
Time :  1627560883.970026 sec
DONE PRODUCTS : 102656
Time :  1627548145.5211046 sec


In [21]:
# KNEIGHBOURS
#train_embed = cupy.load("../input/amazon-ml-challenge-embeddings/train_embed.npy")
#test_embed = cupy.load("../input/amazon-ml-challenge-embeddings/test_embed.npy")

KNN = 50
BATCH_SIZE = 256
NUM_BATCHES = test_embed.shape[0]//BATCH_SIZE
model = NearestNeighbors(n_neighbors=KNN)
model.fit(train_embed)
test_pred = []

start = time.time()
for i in range(NUM_BATCHES + 1):
    start = i*BATCH_SIZE
    end = (i+1)*BATCH_SIZE
    
    distances, indices = model.kneighbors(test_embed[start:end,])
    
    for k in range(end-start):
        IDX = np.where(distances[k,] == np.min(distances[k,]))[0]
        IDS = indices[k,IDX]
        ls = train_df_cpu.iloc[cupy.asnumpy(IDS)].BROWSE_NODE_ID.values
        test_pred.append(ls)
        
    if(i % 50 == 0):
        print("DONE PRODUCTS :", end)
        print("Time : ",int(time.time()-start),"sec")

DONE PRODUCTS : 256
Time :  1627653941 sec
DONE PRODUCTS : 13056
Time :  1627641171 sec
DONE PRODUCTS : 25856
Time :  1627628401 sec
DONE PRODUCTS : 38656
Time :  1627615630 sec
DONE PRODUCTS : 51456
Time :  1627602860 sec
DONE PRODUCTS : 64256
Time :  1627590090 sec
DONE PRODUCTS : 77056
Time :  1627577319 sec
DONE PRODUCTS : 89856
Time :  1627564549 sec
DONE PRODUCTS : 102656
Time :  1627551779 sec


IndexError: Index 183 is out of bounds for axis 0 with size 183

In [27]:
pred_ls = [i[0] for i in test_pred]

result_df = pd.DataFrame.from_dict({
    "PRODUCT_ID" : test_df_cpu["PRODUCT_ID"],
    "BROWSE_NODE_ID" : pred_ls,
})
result_df.to_csv("./submission.csv",index=False)
result_df.head()

,PRODUCT_ID,BROWSE_NODE_ID
0,1,43530
1,2,2321
2,3,13299
3,4,125
4,5,8915
